In [1]:
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv("./LS_2.0.csv")
data.head()

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL\nCASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL\nVOTES,POSTAL\nVOTES,TOTAL\nVOTES,OVER TOTAL ELECTORS \nIN CONSTITUENCY,OVER TOTAL VOTES POLLED \nIN CONSTITUENCY,TOTAL ELECTORS
0,Telangana,ADILABAD,SOYAM BAPU RAO,1,BJP,Lotus,MALE,52,52.0,ST,12th Pass,"Rs 30,99,414\n ~ 30 Lacs+","Rs 2,31,450\n ~ 2 Lacs+",376892,482,377374,25.330684,35.468248,1489790
1,Telangana,ADILABAD,Godam Nagesh,0,TRS,Car,MALE,0,54.0,ST,Post Graduate,"Rs 1,84,77,888\n ~ 1 Crore+","Rs 8,47,000\n ~ 8 Lacs+",318665,149,318814,21.399929,29.964370,1489790
2,Telangana,ADILABAD,RATHOD RAMESH,0,INC,Hand,MALE,3,52.0,ST,12th Pass,"Rs 3,64,91,000\n ~ 3 Crore+","Rs 1,53,00,000\n ~ 1 Crore+",314057,181,314238,21.092771,29.534285,1489790
3,Telangana,ADILABAD,NOTA,0,NOTA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13030,6,13036,0.875023,1.225214,1489790
4,Uttar Pradesh,AGRA,Satyapal Singh Baghel,1,BJP,Lotus,MALE,5,58.0,SC,Doctorate,"Rs 7,42,74,036\n ~ 7 Crore+","Rs 86,06,522\n ~ 86 Lacs+",644459,2416,646875,33.383823,56.464615,1937690


### DATA QUALITY CHECKS & Processing

#### STATE

In [3]:
# State count
data.STATE.unique()

array(['Telangana', 'Uttar Pradesh', 'Maharashtra', 'Gujarat',
       'Rajasthan', 'Kerala', 'West Bengal', 'Uttarakhand',
       'Andhra Pradesh', 'Haryana', 'Punjab', 'Jammu & Kashmir',
       'Andaman & Nicobar Islands', 'Tamil Nadu', 'Bihar',
       'Arunachal Pradesh', 'Odisha', 'Assam', 'Karnataka',
       'Madhya Pradesh', 'Chhattisgarh', 'Chandigarh', 'NCT OF Delhi',
       'Jharkhand', 'Dadra & Nagar Haveli', 'Daman & Diu',
       'Himachal Pradesh', 'Manipur', 'Lakshadweep', 'Mizoram',
       'Nagaland', 'Goa', 'Puducherry', 'Meghalaya', 'Sikkim', 'Tripura'],
      dtype=object)

In [4]:
data.STATE.unique().shape[0]

36

In [5]:
data.STATE = data.STATE.str.upper()
data.STATE.unique().shape[0]

36

#### CONSTITUENCY

In [6]:
data.CONSTITUENCY = data.CONSTITUENCY.str.upper()

In [7]:
data.CONSTITUENCY.isna().any()

False

#### Name

In [8]:
data.NAME.isna().any()

False

In [9]:
### Same candidate from multiple constituencies
for name, group_df in data[data.PARTY != "NOTA"].groupby(["NAME", "PARTY"]):
    if group_df.shape[0] > 1:
        display(group_df)

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL\nCASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL\nVOTES,POSTAL\nVOTES,TOTAL\nVOTES,OVER TOTAL ELECTORS \nIN CONSTITUENCY,OVER TOTAL VOTES POLLED \nIN CONSTITUENCY,TOTAL ELECTORS
64,UTTAR PRADESH,AMETHI,Rahul Gandhi,0,INC,Hand,MALE,6,48.0,GENERAL,Post Graduate,"Rs 15,88,77,063\n ~ 15 Crore+","Rs 72,01,904\n ~ 72 Lacs+",412867,527,413394,23.710378,43.840222,1743515
2247,KERALA,WAYANAD,Rahul Gandhi,1,INC,Hand,MALE,5,48.0,GENERAL,Post Graduate,"Rs 15,88,77,063\n ~ 15 Crore+","Rs 72,01,904\n ~ 72 Lacs+",705034,1333,706367,51.951012,64.640694,1359679


In [10]:
data.iloc[2247, :]

STATE                                                               KERALA
CONSTITUENCY                                                       WAYANAD
NAME                                                          Rahul Gandhi
WINNER                                                                   1
PARTY                                                                  INC
SYMBOL                                                                Hand
GENDER                                                                MALE
CRIMINAL\nCASES                                                          5
AGE                                                                   48.0
CATEGORY                                                           GENERAL
EDUCATION                                                    Post Graduate
ASSETS                                       Rs 15,88,77,063\n ~ 15 Crore+
LIABILITIES                                      Rs 72,01,904\n ~ 72 Lacs+
GENERAL\nVOTES           

In [11]:
data.at[2247, "CRIMINAL\nCASES"] = '6'

In [12]:
# Nota have any valid details
data[data["NAME"] == "NOTA"].iloc[:, 3:13].isna().all()

WINNER             False
PARTY              False
SYMBOL              True
GENDER              True
CRIMINAL\nCASES     True
AGE                 True
CATEGORY            True
EDUCATION           True
ASSETS              True
LIABILITIES         True
dtype: bool

In [13]:
data[data["NAME"] == "NOTA"].WINNER.unique()

array([0])

In [14]:
data[data["NAME"] == "NOTA"].PARTY.unique()

array(['NOTA'], dtype=object)

#### Winner

In [15]:
data.WINNER.isna().any()

False

In [16]:
data.WINNER.unique()

array([1, 0])

In [17]:
# More than 1 winner per constituency
for const, const_df in data.groupby(["STATE","CONSTITUENCY"]):
    if const_df[const_df.WINNER == 1].shape[0] > 1:
        display(const_df)

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL\nCASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL\nVOTES,POSTAL\nVOTES,TOTAL\nVOTES,OVER TOTAL ELECTORS \nIN CONSTITUENCY,OVER TOTAL VOTES POLLED \nIN CONSTITUENCY,TOTAL ELECTORS
170,BIHAR,AURANGABAD,SUSHIL KUMAR SINGH,1,BJP,Lotus,MALE,3,53.0,GENERAL,Post Graduate,"Rs 16,78,95,109\n ~ 16 Crore+","Rs 37,46,607\n ~ 37 Lacs+",426116,1605,427721,24.541005,45.722627,1742883
172,BIHAR,AURANGABAD,NARESH YADAV,1,BSP,Elephant,MALE,5,34.0,GENERAL,Literate,"Rs 1,26,74,177\n ~ 1 Crore+","Rs 8,24,000\n ~ 8 Lacs+",33697,75,33772,1.937709,3.610168,1742883
173,BIHAR,AURANGABAD,UPENDRA PRASAD,0,HAMS,Telephone,MALE,0,55.0,GENERAL,Post Graduate,"Rs 2,03,94,370\n ~ 2 Crore+","Rs 4,71,998\n ~ 4 Lacs+",356846,323,357169,20.492999,38.180741,1742883
177,BIHAR,AURANGABAD,DHIRENDRA KUMAR SINGH,0,IND,Air Conditioner,MALE,0,62.0,GENERAL,Graduate,"Rs 83,89,250\n ~ 83 Lacs+",Rs 0\n ~,24826,6,24832,1.424766,2.654497,1742883
178,BIHAR,AURANGABAD,NOTA,0,NOTA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22397,27,22424,1.286604,2.397086,1742883
179,BIHAR,AURANGABAD,DR DHARMENDRA KUMAR,0,AHFBK,CHESS BOARD,MALE,1,42.0,ST,Graduate,"Rs 60,06,600\n ~ 60 Lacs+",Rs 0\n ~,16584,5,16589,0.951814,1.773335,1742883
180,BIHAR,AURANGABAD,SANTOSH KUMAR SINHA,0,IND,Almirah,MALE,0,43.0,GENERAL,Graduate,"Rs 12,75,000\n ~ 12 Lacs+",Rs 0\n ~,15010,4,15014,0.861446,1.604970,1742883
181,BIHAR,AURANGABAD,AVINASH KUMAR,0,PPID,Table,MALE,0,35.0,GENERAL,Post Graduate,"Rs 37,17,856\n ~ 37 Lacs+",Rs 0\n ~,13668,12,13680,0.784906,1.462368,1742883
182,BIHAR,AURANGABAD,YOGENDRA RAM,0,IND,Balloon,MALE,0,58.0,SC,Literate,"Rs 1,55,000\n ~ 1 Lacs+",Rs 0\n ~,11979,8,11987,0.687768,1.281389,1742883
183,BIHAR,AURANGABAD,SOM PRAKASH,0,SPL,Tractor Chalata Kisan,MALE,0,46.0,GENERAL,Graduate,"Rs 14,37,80,000\n ~ 14 Crore+",Rs 0\n ~,11709,4,11713,0.672047,1.252099,1742883


In [18]:
data[data["NAME"] == "NARESH YADAV"].index

Index([172], dtype='int64')

In [19]:
data.iloc[172, :]

STATE                                                              BIHAR
CONSTITUENCY                                                  AURANGABAD
NAME                                                        NARESH YADAV
WINNER                                                                 1
PARTY                                                                BSP
SYMBOL                                                          Elephant
GENDER                                                              MALE
CRIMINAL\nCASES                                                        5
AGE                                                                 34.0
CATEGORY                                                         GENERAL
EDUCATION                                                       Literate
ASSETS                                       Rs 1,26,74,177\n ~ 1 Crore+
LIABILITIES                                      Rs 8,24,000\n ~ 8 Lacs+
GENERAL\nVOTES                                     

In [20]:
data.at[172, "WINNER"] = 0

#### Party

In [21]:
# Parties
data.PARTY.unique()

array(['BJP', 'TRS', 'INC', 'NOTA', 'BSP', 'NCP', 'VBA', 'APoI', 'CPI(M)',
       'BDJS', 'AITC', 'RSP', 'SP', 'YSRCP', 'TDP', 'JnP', 'INLD', 'SBSP',
       'IND', 'SHS', 'AAP', 'SAD', 'JKN', 'JKPDP', 'JPC', 'DMK', 'PMK',
       'NTK', 'MNM', 'AIADMK', 'RJD', 'CPI(ML)(L)', 'SSD', 'PPA', 'JD(S)',
       'NPEP', 'BMUP', 'BJD', 'AIMIM', 'HAMS', 'AHFBK', 'PPID', 'SPL',
       'ASDC', 'RLD', 'PSPL', 'JD(U)', 'BTP', 'AIFB', 'AGP', 'AIUDF',
       'ABSKP', 'PUNEKP', 'RTORP', 'JNJP', 'LTSP', 'RVNP', 'JANADIP',
       'SDPI', 'DMDK', 'ABGP', 'VCK', 'JMM', 'LIP', 'JDR', 'MOSP', 'MADP',
       'AJPR', 'PMP', 'BBMP', 'AJSUP', 'JVM', 'RMPOI', 'LJP', 'BJKVP',
       'SWP', 'NEINDP', 'RSPSR', 'ravp', 'RSOSP', 'BLSP', 'WPOI',
       'SUCI(C)', 'SJDD', 'ANC', 'JDL', 'VSIP', 'AAM', 'JKP', 'BOPF',
       'UPPL', 'CPIM', 'GGP', 'KEC(M)', 'KEC', 'JAPL', 'AKBMP', 'TJS',
       'IUML', 'BSCP', 'ADAL', 'BRPI', 'MNF', 'PRISMP', 'VPI', 'YKP',
       'NDPP', 'RLTP', 'RAHIS', 'NPF', 'BLSD', 'BVA', 'NAWPP', 'AINRC

In [22]:
data.PARTY.unique().shape

(133,)

In [23]:
# Convert all to Upper case
data.PARTY = data.PARTY.str.upper()
data.PARTY.unique().shape

(133,)

#### Symbol

In [24]:
sorted(data[~data.SYMBOL.isna()].SYMBOL.unique())

['A lady farmer carrying paddy on her head',
 'Air Conditioner',
 'Almirah',
 'Apple',
 'Arrow',
 'Auto- Rickshaw',
 'BELT',
 'BOX',
 'BUCKET',
 'Baby Walker',
 'Balloon',
 'Banana',
 'Bangles',
 'Basket containing Fruits',
 'Bat',
 'Batsman',
 'Battery Torch',
 'Battery torch',
 'Bead Necklace',
 'Bicycle',
 'Black Board',
 'Boat with Man and Sail',
 'Book',
 'Bow & Arrow',
 'Bow and Arrow',
 'Broom',
 'Bungalow',
 'CHAPATI ROLLER',
 'CHESS BOARD',
 'COCONUT FARM',
 'CUTTING PLIERS',
 'Camera',
 'Car',
 'Cauliflower',
 'Ceiling Fan',
 'Chappals',
 'Chimney',
 'Clock',
 'Coat',
 'Cock',
 'Comb',
 'Conch',
 'Cot',
 'Cup & Saucer',
 'DIAMOND',
 'Dao',
 'Dolli',
 'Drum',
 'Ears of Corn And Sickle',
 'Elephant',
 'Envelope',
 'FLUTE',
 'Flag with Three Stars',
 'Flowers and Grass',
 'Football',
 'Football Player',
 'Ganna Kisan',
 'Gas Cylinder',
 'Gas Stove',
 'Gift Pack',
 'Glass Tumbler',
 'Globe',
 'Gramophone',
 'Grapes',
 'HOCKEY & BALL',
 'Hammer, Sickle and Star',
 'Hand',
 'Hand C

In [25]:
data[~data.SYMBOL.isna()].SYMBOL.unique().shape

(126,)

In [26]:
data.SYMBOL = data.SYMBOL.str.upper()
data[~data.SYMBOL.isna()].SYMBOL.unique().shape

(125,)

#### Gender

In [27]:
# Gender
data.GENDER.unique()

array(['MALE', nan, 'FEMALE'], dtype=object)

#### Criminal Cases

In [28]:
data[data["CRIMINAL\nCASES"].isna()].NAME.unique()

array(['NOTA'], dtype=object)

#### AGE

In [29]:
# Convert the age column to int from float
sorted(data.AGE.unique())

[25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 52.0,
 54.0,
 nan,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 53.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 86.0]

In [30]:
data.AGE.fillna(value=0, inplace=True)

/tmp/ipykernel_19777/3791563789.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.AGE.fillna(value=0, inplace=True)


In [31]:
data.AGE.isna().any()

False

In [32]:
data.AGE = data.AGE.astype(int)
data.AGE

0       52
1       54
2       52
3        0
4       58
        ..
2258    43
2259    63
2260    49
2261    47
2262     0
Name: AGE, Length: 2263, dtype: int64

#### Category

In [33]:
# Category
data.CATEGORY.unique()

array(['ST', nan, 'SC', 'GENERAL'], dtype=object)

#### Education

In [34]:
data.EDUCATION.unique()

array(['12th Pass', 'Post Graduate', nan, 'Doctorate', 'Graduate',
       'Others', '10th Pass', '8th Pass', 'Graduate Professional',
       'Literate', 'Illiterate', '5th Pass', 'Not Available',
       'Post Graduate\n'], dtype=object)

In [35]:
data.EDUCATION = data.EDUCATION.str.replace('Post Graduate\n', "Post Graduate").fillna("Not Available").replace("Others", "Not Available").replace("Graduate Professional", 
                                                                                                                                                   "Graduate")

In [36]:
def is_literate(ele):
    if ele == "Not Available":
        return 'Not Available'
    elif ele == "Illiterate":
        return "Illiterate"
    else:
        return "Literate"
        
data["LITERATE"] = data.EDUCATION.apply(is_literate)

In [37]:
data[data.LITERATE == "Illiterate"]

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL\nCASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL\nVOTES,POSTAL\nVOTES,TOTAL\nVOTES,OVER TOTAL ELECTORS \nIN CONSTITUENCY,OVER TOTAL VOTES POLLED \nIN CONSTITUENCY,TOTAL ELECTORS,LITERATE
102,JAMMU & KASHMIR,ANANTNAG,SOFI YOUSUF,0,BJP,LOTUS,MALE,1,53,GENERAL,Illiterate,"Rs 1,17,10,000\n ~ 1 Crore+",Rs 0\n ~,9687,538,10225,0.731518,8.149683,1397779,Illiterate
144,ANDHRA PRADESH,ARUKU,KANGALA BALUDORA,0,IND,CUP & SAUCER,MALE,0,61,ST,Illiterate,"Rs 3,40,500\n ~ 3 Lacs+",Rs 0\n ~,13810,16,13826,0.952586,1.282281,1451418,Illiterate
413,TELANGANA,BHONGIR,Singapaka Lingam,0,IND,ROAD ROLLER,MALE,0,44,SC,Illiterate,Nil,Rs 0\n ~,27973,0,27973,1.718208,2.306525,1628033,Illiterate
653,TAMIL NADU,DINDIGUL,"VELUSAMY, P.",1,DMK,RISING SUN,MALE,0,52,GENERAL,Illiterate,"Rs 14,80,76,407\n ~ 14 Crore+","Rs 3,14,74,526\n ~ 3 Crore+",743813,2710,746523,48.416382,64.303822,1541881,Illiterate
2218,TAMIL NADU,VILUPPURAM,ARASAN K,0,IND,JACKFRUIT,MALE,0,64,SC,Illiterate,"Rs 3,30,000\n ~ 3 Lacs+",Rs 0\n ~,12778,3,12781,0.884906,1.124991,1444335,Illiterate


#### Assets

In [38]:
data[data.ASSETS.isna()]

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL\nCASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL\nVOTES,POSTAL\nVOTES,TOTAL\nVOTES,OVER TOTAL ELECTORS \nIN CONSTITUENCY,OVER TOTAL VOTES POLLED \nIN CONSTITUENCY,TOTAL ELECTORS,LITERATE
3,TELANGANA,ADILABAD,NOTA,0,NOTA,NaN,NaN,NaN,0,NaN,Not Available,NaN,NaN,13030,6,13036,0.875023,1.225214,1489790,Not Available
14,GUJARAT,AHMEDABAD WEST,NOTA,0,NOTA,NaN,NaN,NaN,0,NaN,Not Available,NaN,NaN,14580,139,14719,0.895688,1.473030,1643317,Not Available
39,WEST BENGAL,ALIPURDUARS,NOTA,0,NOTA,NaN,NaN,NaN,0,NaN,Not Available,NaN,NaN,21147,28,21175,1.284592,1.533114,1648383,Not Available
46,UTTARAKHAND,ALMORA,NOTA,0,NOTA,NaN,NaN,NaN,0,NaN,Not Available,NaN,NaN,15311,194,15505,1.158985,2.215611,1337808,Not Available
54,ANDHRA PRADESH,AMALAPURAM,NOTA,0,NOTA,NaN,NaN,NaN,0,NaN,Not Available,NaN,NaN,16427,41,16468,1.128288,1.333044,1459556,Not Available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,TAMIL NADU,VIRUDHUNAGAR,NOTA,0,NOTA,NaN,NaN,NaN,0,NaN,Not Available,NaN,NaN,17087,205,17292,1.165028,1.607174,1484256,Not Available
2230,ANDHRA PRADESH,VISAKHAPATNAM,NOTA,0,NOTA,NaN,NaN,NaN,0,NaN,Not Available,NaN,NaN,16626,20,16646,0.909966,1.342505,1829300,Not Available
2235,ANDHRA PRADESH,VIZIANAGARAM,NOTA,0,NOTA,NaN,NaN,NaN,0,NaN,Not Available,NaN,NaN,29468,33,29501,1.961529,2.413302,1503980,Not Available
2241,TELANGANA,WARANGAL,NOTA,0,NOTA,NaN,NaN,NaN,0,NaN,Not Available,NaN,NaN,18764,37,18801,1.127990,1.770886,1666770,Not Available


In [39]:
data.ASSETS.fillna("", inplace=True)

/tmp/ipykernel_19777/1303570051.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.ASSETS.fillna("", inplace=True)


In [40]:
data.ASSETS = data.ASSETS.str.replace("Nil", "0")

In [41]:
def assets_to_num(ele):
    split_values = str(ele).split("~")[0].strip().replace(",", "").split()
    if len(split_values) > 0:
        return split_values[-1]
    return 0

In [42]:
# ASSETS
data.ASSETS = data.ASSETS.apply(assets_to_num)

In [43]:
data.ASSETS

0          3099414
1         18477888
2         36491000
3                0
4         74274036
           ...    
2258       4890000
2259    1287851556
2260     903663001
2261      58577327
2262             0
Name: ASSETS, Length: 2263, dtype: object

#### LIABILITIES

In [44]:
data.LIABILITIES.fillna("", inplace=True)

/tmp/ipykernel_19777/1634678826.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.LIABILITIES.fillna("", inplace=True)


In [45]:
data.LIABILITIES = data.LIABILITIES.str.replace("Nil", "0")

In [46]:
data.LIABILITIES = data.LIABILITIES.apply(assets_to_num)
data.LIABILITIES

0         231450
1         847000
2       15300000
3              0
4        8606522
          ...   
2258     1020000
2259    11535000
2260           0
2261     5250000
2262           0
Name: LIABILITIES, Length: 2263, dtype: object

#### Last Columns

In [47]:
# Last numerical columns
data.iloc[:, 13:].isna().any()

GENERAL\nVOTES                               False
POSTAL\nVOTES                                False
TOTAL\nVOTES                                 False
OVER TOTAL ELECTORS \nIN CONSTITUENCY        False
OVER TOTAL VOTES POLLED \nIN CONSTITUENCY    False
TOTAL ELECTORS                               False
LITERATE                                     False
dtype: bool

#### Columns Names

In [48]:
data.columns = ["_".join(col.split()) for col in data.columns]

#### After Cleaning

In [49]:
data.head()

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL_CASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL_VOTES,POSTAL_VOTES,TOTAL_VOTES,OVER_TOTAL_ELECTORS_IN_CONSTITUENCY,OVER_TOTAL_VOTES_POLLED_IN_CONSTITUENCY,TOTAL_ELECTORS,LITERATE
0,TELANGANA,ADILABAD,SOYAM BAPU RAO,1,BJP,LOTUS,MALE,52,52,ST,12th Pass,3099414,231450,376892,482,377374,25.330684,35.468248,1489790,Literate
1,TELANGANA,ADILABAD,Godam Nagesh,0,TRS,CAR,MALE,0,54,ST,Post Graduate,18477888,847000,318665,149,318814,21.399929,29.964370,1489790,Literate
2,TELANGANA,ADILABAD,RATHOD RAMESH,0,INC,HAND,MALE,3,52,ST,12th Pass,36491000,15300000,314057,181,314238,21.092771,29.534285,1489790,Literate
3,TELANGANA,ADILABAD,NOTA,0,NOTA,NaN,NaN,NaN,0,NaN,Not Available,0,0,13030,6,13036,0.875023,1.225214,1489790,Not Available
4,UTTAR PRADESH,AGRA,Satyapal Singh Baghel,1,BJP,LOTUS,MALE,5,58,SC,Doctorate,74274036,8606522,644459,2416,646875,33.383823,56.464615,1937690,Literate


#### Data Type

In [50]:
def numerical_parsing(ele, _type=int):
    try:
        return _type(ele)
    except Exception as err:
        return -1

In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2263 entries, 0 to 2262
Data columns (total 20 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   STATE                                    2263 non-null   object 
 1   CONSTITUENCY                             2263 non-null   object 
 2   NAME                                     2263 non-null   object 
 3   WINNER                                   2263 non-null   int64  
 4   PARTY                                    2263 non-null   object 
 5   SYMBOL                                   2018 non-null   object 
 6   GENDER                                   2018 non-null   object 
 7   CRIMINAL_CASES                           2018 non-null   object 
 8   AGE                                      2263 non-null   int64  
 9   CATEGORY                                 2018 non-null   object 
 10  EDUCATION                                2263 no

In [52]:
data[data.ASSETS.isna()]

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL_CASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL_VOTES,POSTAL_VOTES,TOTAL_VOTES,OVER_TOTAL_ELECTORS_IN_CONSTITUENCY,OVER_TOTAL_VOTES_POLLED_IN_CONSTITUENCY,TOTAL_ELECTORS,LITERATE


In [53]:
data[data.ASSETS == "Nil"]

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL_CASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL_VOTES,POSTAL_VOTES,TOTAL_VOTES,OVER_TOTAL_ELECTORS_IN_CONSTITUENCY,OVER_TOTAL_VOTES_POLLED_IN_CONSTITUENCY,TOTAL_ELECTORS,LITERATE


In [54]:
data[data.ASSETS == "Available"]

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL_CASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL_VOTES,POSTAL_VOTES,TOTAL_VOTES,OVER_TOTAL_ELECTORS_IN_CONSTITUENCY,OVER_TOTAL_VOTES_POLLED_IN_CONSTITUENCY,TOTAL_ELECTORS,LITERATE
468,BIHAR,BUXAR,Ramchandra Singh Yadav,0,IND,ALMIRAH,MALE,Not Available,42,GENERAL,Not Available,Available,Available,10721,9,10730,0.586540,1.087175,1829373,Not Available
532,TAMIL NADU,CHIDAMBARAM,SIVAJOTHI M,0,NTK,GANNA KISAN,MALE,Not Available,35,SC,Not Available,Available,Available,37329,142,37471,2.531445,3.246331,1480222,Not Available
612,UTTAR PRADESH,DEORIA,BINOD KUMAR JAISWAL,0,BSP,ELEPHANT,MALE,Not Available,56,GENERAL,Not Available,Available,Available,327728,2985,330713,18.852693,32.563441,1754195,Not Available
613,UTTAR PRADESH,DEORIA,NIYAZ AHMED,0,INC,HAND,MALE,Not Available,57,GENERAL,Not Available,Available,Available,50749,307,51056,2.910509,5.027196,1754195,Not Available
654,TAMIL NADU,DINDIGUL,"JOTHIMUTHU, K.",0,PMK,MANGO,MALE,Not Available,48,GENERAL,Not Available,Available,Available,206782,769,207551,13.460896,17.877979,1541881,Not Available
656,TAMIL NADU,DINDIGUL,"MANSOORALI KHAN, A.",0,NTK,GANNA KISAN,MALE,Not Available,56,GENERAL,Not Available,Available,Available,54574,383,54957,3.564283,4.733873,1541881,Not Available
688,ANDHRA PRADESH,ELURU,PENTAPATI PULLARAO,0,JNP,GLASS TUMBLER,MALE,Not Available,71,GENERAL,Not Available,Available,Available,76481,346,76827,4.816891,5.766884,1594950,Not Available
972,BIHAR,JAMUI (SC),UPENDRA RAVIDAS,0,BSP,ELEPHANT,MALE,Not Available,40,SC,Not Available,Available,Available,31504,107,31611,1.839117,3.329012,1718814,Not Available
1033,UTTAR PRADESH,KAISERGANJ,Vinay Kumar Pandey 'Vinnu',0,INC,HAND,MALE,Not Available,54,GENERAL,Not Available,Available,Available,37049,83,37132,2.056097,3.780019,1805946,Not Available
1052,TAMIL NADU,KALLAKURICHI,Sharfudeen S,0,NTK,GANNA KISAN,MALE,Not Available,40,GENERAL,Not Available,Available,Available,30103,143,30246,1.978236,2.510177,1528938,Not Available


In [55]:
data.ASSETS = data.ASSETS.apply(numerical_parsing, args=(int,))
data.LIABILITIES = data.LIABILITIES.apply(numerical_parsing, args=(int,))
data.CRIMINAL_CASES = data.CRIMINAL_CASES.apply(numerical_parsing, args=(int,))

In [56]:
data.ASSETS = data.ASSETS.astype(int)
data.LIABILITIES = data.LIABILITIES.astype(int)
data.CRIMINAL_CASES = data.CRIMINAL_CASES.astype(int)

In [57]:
data.head()

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL_CASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL_VOTES,POSTAL_VOTES,TOTAL_VOTES,OVER_TOTAL_ELECTORS_IN_CONSTITUENCY,OVER_TOTAL_VOTES_POLLED_IN_CONSTITUENCY,TOTAL_ELECTORS,LITERATE
0,TELANGANA,ADILABAD,SOYAM BAPU RAO,1,BJP,LOTUS,MALE,52,52,ST,12th Pass,3099414,231450,376892,482,377374,25.330684,35.468248,1489790,Literate
1,TELANGANA,ADILABAD,Godam Nagesh,0,TRS,CAR,MALE,0,54,ST,Post Graduate,18477888,847000,318665,149,318814,21.399929,29.964370,1489790,Literate
2,TELANGANA,ADILABAD,RATHOD RAMESH,0,INC,HAND,MALE,3,52,ST,12th Pass,36491000,15300000,314057,181,314238,21.092771,29.534285,1489790,Literate
3,TELANGANA,ADILABAD,NOTA,0,NOTA,NaN,NaN,-1,0,NaN,Not Available,0,0,13030,6,13036,0.875023,1.225214,1489790,Not Available
4,UTTAR PRADESH,AGRA,Satyapal Singh Baghel,1,BJP,LOTUS,MALE,5,58,SC,Doctorate,74274036,8606522,644459,2416,646875,33.383823,56.464615,1937690,Literate


In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2263 entries, 0 to 2262
Data columns (total 20 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   STATE                                    2263 non-null   object 
 1   CONSTITUENCY                             2263 non-null   object 
 2   NAME                                     2263 non-null   object 
 3   WINNER                                   2263 non-null   int64  
 4   PARTY                                    2263 non-null   object 
 5   SYMBOL                                   2018 non-null   object 
 6   GENDER                                   2018 non-null   object 
 7   CRIMINAL_CASES                           2263 non-null   int64  
 8   AGE                                      2263 non-null   int64  
 9   CATEGORY                                 2018 non-null   object 
 10  EDUCATION                                2263 no

In [59]:
data[(data.apply(lambda ele: ele == -1).any(axis=1)) & (data.NAME != "NOTA")]

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL_CASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL_VOTES,POSTAL_VOTES,TOTAL_VOTES,OVER_TOTAL_ELECTORS_IN_CONSTITUENCY,OVER_TOTAL_VOTES_POLLED_IN_CONSTITUENCY,TOTAL_ELECTORS,LITERATE
186,ASSAM,AUTONOMOUS DISTRICT,JONES INGTI KATHAR,0,IND,BATTERY TORCH,MALE,0,68,ST,Post Graduate,610218,-1,39425,158,39583,4.973082,6.406488,795945,Literate
468,BIHAR,BUXAR,Ramchandra Singh Yadav,0,IND,ALMIRAH,MALE,-1,42,GENERAL,Not Available,-1,-1,10721,9,10730,0.586540,1.087175,1829373,Not Available
532,TAMIL NADU,CHIDAMBARAM,SIVAJOTHI M,0,NTK,GANNA KISAN,MALE,-1,35,SC,Not Available,-1,-1,37329,142,37471,2.531445,3.246331,1480222,Not Available
612,UTTAR PRADESH,DEORIA,BINOD KUMAR JAISWAL,0,BSP,ELEPHANT,MALE,-1,56,GENERAL,Not Available,-1,-1,327728,2985,330713,18.852693,32.563441,1754195,Not Available
613,UTTAR PRADESH,DEORIA,NIYAZ AHMED,0,INC,HAND,MALE,-1,57,GENERAL,Not Available,-1,-1,50749,307,51056,2.910509,5.027196,1754195,Not Available
654,TAMIL NADU,DINDIGUL,"JOTHIMUTHU, K.",0,PMK,MANGO,MALE,-1,48,GENERAL,Not Available,-1,-1,206782,769,207551,13.460896,17.877979,1541881,Not Available
656,TAMIL NADU,DINDIGUL,"MANSOORALI KHAN, A.",0,NTK,GANNA KISAN,MALE,-1,56,GENERAL,Not Available,-1,-1,54574,383,54957,3.564283,4.733873,1541881,Not Available
688,ANDHRA PRADESH,ELURU,PENTAPATI PULLARAO,0,JNP,GLASS TUMBLER,MALE,-1,71,GENERAL,Not Available,-1,-1,76481,346,76827,4.816891,5.766884,1594950,Not Available
972,BIHAR,JAMUI (SC),UPENDRA RAVIDAS,0,BSP,ELEPHANT,MALE,-1,40,SC,Not Available,-1,-1,31504,107,31611,1.839117,3.329012,1718814,Not Available
1033,UTTAR PRADESH,KAISERGANJ,Vinay Kumar Pandey 'Vinnu',0,INC,HAND,MALE,-1,54,GENERAL,Not Available,-1,-1,37049,83,37132,2.056097,3.780019,1805946,Not Available


In [60]:
data[(data.isna().any(axis=1)) & (data.NAME != "NOTA")]

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL_CASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL_VOTES,POSTAL_VOTES,TOTAL_VOTES,OVER_TOTAL_ELECTORS_IN_CONSTITUENCY,OVER_TOTAL_VOTES_POLLED_IN_CONSTITUENCY,TOTAL_ELECTORS,LITERATE


In [61]:
data.to_csv("./modified_dataset.csv", index=False)